In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime

In [17]:
train_filepath = 'data/train.csv'
train_df = pd.read_table(train_filepath)
test_filepath = 'data/test.csv'
test_df = pd.read_table(test_filepath)

/Users/zhangge/Documents/DB_proj/project-retepognajd/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/Users/zhangge/Documents/DB_proj/project-retepognajd/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


In [62]:
# train_df.columns
test_df.head()

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,10039,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP01,1,2/300*X = 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10482,52vEY7f17k,"Unit CTA1_06, Section CTA1_06-3",FOR05,1,xScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,28
2,11024,52vEY7f17k,"Unit ES_03, Section ES_03-6",EG52,1,2.9 = -2x+3.7+x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SkillRule: Consolidate vars, no coeff; CLT]",27
3,11476,52vEY7f17k,"Unit CTA1_08, Section CTA1_08-3",REAL10,1,yScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,65
4,11948,52vEY7f17k,"Unit ES_04, Section ES_04-12",EG58,7,xR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
hierarchy = train_df['Problem Hierarchy']

units, sections = [], []
for i in range(len(hierarchy)):
    units.append(hierarchy[i].split(',')[0].strip())
    sections.append(hierarchy[i].split(',')[1].strip())

train_df['Problem Unit'] = pd.Series(units, index=train_df.index)
train_df['Problem Section'] = pd.Series(sections, index=train_df.index)

cat = 'Problem Hierarchy'
prhs = list(set(train_df[cat]))
prh_dict = {}
for idx,prh in enumerate(prhs):
    prh_dict[idx] = prh
    train_df.loc[train_df[cat] == prh,cat] = idx

cat = 'Problem Unit'
prus = list(set(train_df[cat]))
pru_dict = {}
for idx,pru in enumerate(prus):
    pru_dict[idx] = pru
    train_df.loc[train_df[cat] == pru,cat] = idx

cat = 'Problem Name'
prns = list(set(train_df[cat]))
prn_dict = {}
for idx,prn in enumerate(prns):
    prn_dict[idx] = prn
    train_df.loc[train_df[cat] == prn,cat] = idx

In [57]:
def cfa_general_row_test(row,df,col1,col2):

    n1 = row[col1]
    n2 = row[col2]
    print(n1,n2)
    try:
        c = df.loc[n1].loc[n2].loc[1]
        
    except KeyError:
        return 0
    
    try:
        ic = df.loc[n1].loc[n2].loc[0]
    except KeyError:
        ic = 0 
    return c / (c + ic)

def cfa_general_row_train(row,df,col1,col2):
    n1 = row[col1]
    n2 = row[col2]
    t = row["Row"]
    print(t)
    if 1 in df.loc[n1,n2].index:
        c = df.loc[n1,n2,1]
    else:
        return 0
    
    if 0 in df.loc[n1,n2].index:
        ic = df.loc[n1,n2,0]
    else:
        ic = 0
    return c / (c + ic)


def gen_cfa(traindata,testdata,col1,col2):
    cfa_tb = traindata[[col1,col2,'Correct First Attempt',]]\
                .groupby([col1,col2,'Correct First Attempt']).size()
    cfa_tb.head()
#     new_col_name = 'CFA|' + ','.join([col1,col2])
#     train_newcol = traindata.apply(cfa_general_row_train,args = (cfa_tb,col1,col2),axis=1)
    test_newcol = testdata.apply(cfa_general_row_test,args = (cfa_tb,col1,col2),axis=1)
    return (train_newcol,test_newcol)

In [58]:
a,b = gen_cfa(train_df,test_df,'Anon Student Id','Problem Unit')

KeyError: ('Problem Unit', 'occurred at index 0')

In [29]:
c =a

NameError: name 'a' is not defined